## CS 238B: Hamiltonian Simulation in Qiskit

In [4]:
from qiskit import *
import numpy as np

In [18]:
# Example 1: Hamiltonian H = gj * Xj, where gj is a constant and Xj is the Pauli-X operator acting on qubit j

# Define the number of qubits
n = 3

# Construct the circuit for this Hamiltonian simulation using 3 qubits
q = QuantumRegister(n)
c = ClassicalRegister(n)
circuit = QuantumCircuit(q, c)

# Define the constants
g = 1.0
t = 1.0

# Apply the following sequence of gates to each qubit j: Hadamard H gate, RZ gate with angle 2 * g * t, Hadamard H gate
for j in range(n):
    circuit.h(q[j])
    circuit.rz(2 * g * t, q[j])
    circuit.h(q[j])

# Print the circuit
print(circuit)

# Simulate the circuit
sim = Aer.get_backend('unitary_simulator')
job = sim.run(transpile(circuit,backend=sim), shots=1)
result = job.result()
print(result.get_unitary(circuit, decimals=3))



       ┌───┐┌───────┐┌───┐
q31_0: ┤ H ├┤ Rz(2) ├┤ H ├
       ├───┤├───────┤├───┤
q31_1: ┤ H ├┤ Rz(2) ├┤ H ├
       ├───┤├───────┤├───┤
q31_2: ┤ H ├┤ Rz(2) ├┤ H ├
       └───┘└───────┘└───┘
 c8: 3/═══════════════════
                          
Operator([[ 0.158-0.j   , -0.   -0.246j, -0.   -0.246j, -0.383+0.j   ,
           -0.   -0.246j, -0.383+0.j   , -0.383+0.j   ,  0.   +0.596j],
          [-0.   -0.246j,  0.158-0.j   , -0.383+0.j   , -0.   -0.246j,
           -0.383+0.j   , -0.   -0.246j,  0.   +0.596j, -0.383+0.j   ],
          [-0.   -0.246j, -0.383+0.j   ,  0.158-0.j   , -0.   -0.246j,
           -0.383+0.j   ,  0.   +0.596j, -0.   -0.246j, -0.383+0.j   ],
          [-0.383+0.j   , -0.   -0.246j, -0.   -0.246j,  0.158-0.j   ,
            0.   +0.596j, -0.383+0.j   , -0.383+0.j   ,  0.   -0.246j],
          [-0.   -0.246j, -0.383+0.j   , -0.383+0.j   ,  0.   +0.596j,
            0.158-0.j   , -0.   -0.246j, -0.   -0.246j, -0.383+0.j   ],
          [-0.383+0.j   , -0.   -0.246j,  

## Correctness verification for example 1



In [19]:
from scipy.linalg import expm

# Define the Identity matrix
I = np.eye(2)

# Define the Pauli-X matrix
X = np.array([[0, 1], [1, 0]])

# Calculute X_0 = X⊗I⊗I
X0 = kron(X, kron(I, I))

# Calculate X_1 = I⊗X⊗I
X1 = kron(I, kron(X, I))

# Calculate X_2 = I⊗I⊗X
X2 = kron(I, kron(I, X))

# The Hamiltonian matrix is obtained by summing the three tensor products
H = g * (X0 + X1 + X2) # g = 1

print(H)

unitary = expm(H * -1j)

# round the entries for better printing
unitary = np.round(unitary, 3)
print(unitary)





[[0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 1. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1. 1. 0.]]
[[ 0.158+0.j     0.   -0.246j  0.   -0.246j -0.383+0.j     0.   -0.246j
  -0.383+0.j    -0.383+0.j     0.   +0.596j]
 [ 0.   -0.246j  0.158+0.j    -0.383+0.j     0.   -0.246j -0.383+0.j
   0.   -0.246j  0.   +0.596j -0.383+0.j   ]
 [ 0.   -0.246j -0.383+0.j     0.158+0.j     0.   -0.246j -0.383+0.j
   0.   +0.596j  0.   -0.246j -0.383+0.j   ]
 [-0.383+0.j     0.   -0.246j  0.   -0.246j  0.158+0.j     0.   +0.596j
  -0.383+0.j    -0.383+0.j     0.   -0.246j]
 [ 0.   -0.246j -0.383+0.j    -0.383+0.j     0.   +0.596j  0.158+0.j
   0.   -0.246j  0.   -0.246j -0.383+0.j   ]
 [-0.383+0.j     0.   -0.246j  0.   +0.596j -0.383+0.j     0.   -0.246j
   0.158+0.j    -0.383+0.j     0.   -0.246j]
 [-0.383+0.j     0.   +0.596j  0.   -0.246j -0.383+0.j     0.   -0.246j
  -0.383+0.j     0

In [20]:
# Example 2: Hamiltonian H = gj * Xj - Zj * Zj+1, where gj is a constant, Xj is the Pauli-X operator acting on qubit j, and Zj is the Pauli-Z operator acting on qubit j

# Define the number of qubits
n = 3

# Construct the circuit for this Hamiltonian simulation using 3 qubits
q = QuantumRegister(n)
c = ClassicalRegister(n)

circuit = QuantumCircuit(q, c)

# Define the constants
g = 1.0
t = 1.0

# Apply the following sequence of gates to qubit 1: Hadamard H gate, RZ gate with angle 2 * g * t, Hadamard H gate
circuit.h(q[0])
circuit.rz(2 * g * t, q[0])
circuit.h(q[0])

# Apply controlled cnot gates to qubits 1 and 2
circuit.cx(q[0], q[1])

# Apply RZ gate with angle -2 * g * t to qubit 2
circuit.rz(-2 * g * t, q[1])

# Apply controlled cnot gates to qubits 1 and 2
circuit.cx(q[0], q[1])

# Apply the following sequence of gates to qubit 2: Hadamard H gate, RZ gate with angle 2 * g * t, Hadamard H gate
circuit.h(q[1])
circuit.rz(2 * g * t, q[1])
circuit.h(q[1])

# Apply controlled cnot gates to qubits 2 and 3
circuit.cx(q[1], q[2])

# Apply RZ gate with angle -2 * g * t to qubit 3
circuit.rz(-2 * g * t, q[2])


       ┌───┐┌───────┐┌───┐                                            »
q34_0: ┤ H ├┤ Rz(2) ├┤ H ├──■──────────────■──────────────────────────»
       └───┘└───────┘└───┘┌─┴─┐┌────────┐┌─┴─┐┌───┐┌───────┐┌───┐     »
q34_1: ───────────────────┤ X ├┤ Rz(-2) ├┤ X ├┤ H ├┤ Rz(2) ├┤ H ├──■──»
                          └───┘└────────┘└───┘└───┘└───────┘└───┘┌─┴─┐»
q34_2: ──────────────────────────────────────────────────────────┤ X ├»
                                                                 └───┘»
 c9: 3/═══════════════════════════════════════════════════════════════»
                                                                      »
«                      
«q34_0: ───────────────
«                      
«q34_1: ────────────■──
«       ┌────────┐┌─┴─┐
«q34_2: ┤ Rz(-2) ├┤ X ├
«       └────────┘└───┘
« c9: 3/═══════════════
«                      
Operator([[-0.121+0.265j,  0.413+0.189j, -0.   -0.455j, -0.708+0.j   ,
            0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.

## Correctness verification for example 2

In [26]:
X0 = kron(X, kron(I, I))
X1 = kron(I, kron(X, I))

# Define the Pauli-Z matrix
Z = np.array([[1, 0], [0, -1]])

# Calculate Z_0 = Z⊗I⊗I
Z0 = kron(Z, kron(I, I))

# Calculate Z_1 = I⊗Z⊗I
Z1 = kron(I, kron(Z, I))

# Calculate Z_2 = I⊗I⊗Z
Z2 = kron(I, kron(I, Z))

negZ0Z1 = -1 * np.matmul(Z0, Z1)
negZ1Z2 = -1 * np.matmul(Z1, Z2)

H = X0 + negZ0Z1 + X1 + negZ1Z2

unitary = expm(H * -1j)

# round the entries for better printing
unitary = np.round(unitary, 3)
print("Calculated Unitary\n", unitary)

Calculated Unitary
 [[-0.566+0.394j  0.   +0.j     0.   -0.086j  0.   +0.j     0.48 -0.086j
   0.   +0.j    -0.48 -0.222j  0.   +0.j   ]
 [ 0.   +0.j     0.393+0.222j  0.   +0.j     0.   -0.529j  0.   +0.j
  -0.48 -0.086j  0.   +0.j    -0.48 -0.222j]
 [ 0.   -0.086j  0.   +0.j    -0.566-0.394j  0.   +0.j    -0.48 +0.222j
   0.   +0.j    -0.48 -0.086j  0.   +0.j   ]
 [ 0.   +0.j     0.   -0.529j  0.   +0.j     0.393-0.222j  0.   +0.j
  -0.48 +0.222j  0.   +0.j     0.48 -0.086j]
 [ 0.48 -0.086j  0.   +0.j    -0.48 +0.222j  0.   +0.j     0.393-0.222j
   0.   +0.j     0.   -0.529j  0.   +0.j   ]
 [ 0.   +0.j    -0.48 -0.086j  0.   +0.j    -0.48 +0.222j  0.   +0.j
  -0.566-0.394j  0.   +0.j     0.   -0.086j]
 [-0.48 -0.222j  0.   +0.j    -0.48 -0.086j  0.   +0.j    -0.   -0.529j
   0.   +0.j     0.393+0.222j  0.   +0.j   ]
 [ 0.   +0.j    -0.48 -0.222j  0.   +0.j     0.48 -0.086j  0.   +0.j
   0.   -0.086j  0.   +0.j    -0.566+0.394j]]
